In [29]:
import os
import re
import pandas as pd

In [30]:
import os
import re
import pandas as pd

folder_path = '../Simulation/Assets/Results/Correlation_cost_completiontime'

column_name = ['File_name', 'Type',  'C_max', 'C_avg']
values = []

# get folder
for folder_name in os.listdir(folder_path):
    if not folder_name.endswith('meta'):
        cost_type = folder_name.split('_')[-1]
        if not cost_type[0].isupper():
            cost_type = cost_type[0].upper() + cost_type[1:]

        # get files in folder
        folder_full_path = os.path.join(folder_path, folder_name)

        # Iterate over files in the folder
        for file_name in os.listdir(folder_full_path):
            if not file_name.endswith('meta'):
                # 정규 표현식을 사용하여 'prev'와 'now'를 기준으로 파일 이름 분리
                match = re.match(r'result_(prev.*?)_with_(now.*?)\.csv', file_name)
                if match:
                    prev_part  = match.group(1)
                    now_part = match.group(2) + '.csv'

                    # "RoutePoints"를 "Truck"으로 바꿈
                    prev_file_name = prev_part.replace("RoutePoints", "Truck")
                    now_file_name = now_part.replace("RoutePoints", "Truck")

                    # read csv file
                    file_full_path = os.path.join(folder_full_path, file_name)
                    df = pd.read_csv(file_full_path)
                    # print(df.columns)
                    c_max_prev = df['C_max_prev'][0]
                    c_max_now = df['C_max_now'][0]

                    c_avg_prev = df.loc[df['Truck_id'].str[6:].astype(int)<100, 'Completion_Time'].mean()
                    c_avg_now = df.loc[df['Truck_id'].str[6:].astype(int)>=100, 'Completion_Time'].mean()

                    # Append values to the list
                    prev_value = [prev_file_name, cost_type, c_max_prev, c_avg_prev]
                    values.append(prev_value)
                    now_value = [now_file_name, cost_type, c_max_now, c_avg_now]
                    values.append(now_value)

# create dataframe
cost_type_df = pd.DataFrame(values, columns=column_name)
print(cost_type_df)


                              File_name Type    C_max     C_avg
0    prev_Truck_20_LP_0_0_100_10rep.csv  Max  505.960  428.0428
1     now_Truck_20_LP_0_0_100_10rep.csv  Max  483.008  437.2152
2    prev_Truck_20_LP_0_0_100_11rep.csv  Max  471.872  427.3252
3     now_Truck_20_LP_0_0_100_11rep.csv  Max  495.936  436.5496
4    prev_Truck_20_LP_0_0_100_12rep.csv  Max  609.776  519.2600
..                                  ...  ...      ...       ...
235   now_Truck_20_LP_40_50_10_7rep.csv  Min  447.488  384.7324
236  prev_Truck_20_LP_40_50_10_8rep.csv  Min  434.984  378.5184
237   now_Truck_20_LP_40_50_10_8rep.csv  Min  413.224  376.2884
238  prev_Truck_20_LP_40_50_10_9rep.csv  Min  451.840  386.1620
239   now_Truck_20_LP_40_50_10_9rep.csv  Min  453.392  377.7228

[240 rows x 4 columns]


In [31]:
cost_type_df[cost_type_df['File_name'] == 'prev_Truck_20_LP_0_0_100_1rep.csv']

,File_name,Type,C_max,C_avg
20,prev_Truck_20_LP_0_0_100_1rep.csv,Max,521.536,452.7484
100,prev_Truck_20_LP_0_0_100_1rep.csv,Middle,466.880,428.1868
180,prev_Truck_20_LP_0_0_100_1rep.csv,Min,427.120,384.7000


In [40]:
original_file_name = 'C:\\Users\\USER\\workspace\\TSB\\Model\\corrlation_data.csv'

original_df = pd.read_csv(original_file_name)


# type열 공백제거
original_df['Type'] = original_df['Type'].str.strip()
print(original_df)

                               File_name    Type  Objective_value
0      prev_Truck_20_LP_0_0_100_1rep.csv     Min            63800
1       now_Truck_20_LP_0_0_100_1rep.csv     Min            68000
2     prev_Truck_20_LP_40_50_10_1rep.csv     Min            34751
3      now_Truck_20_LP_40_50_10_1rep.csv     Min            34924
4      prev_Truck_20_LP_0_0_100_1rep.csv  Middle            98400
..                                   ...     ...              ...
235   now_Truck_20_LP_40_50_10_20rep.csv  Middle            48319
236   prev_Truck_20_LP_0_0_100_20rep.csv     Max           112800
237    now_Truck_20_LP_0_0_100_20rep.csv     Max           115600
238  prev_Truck_20_LP_40_50_10_20rep.csv     Max            59834
239   now_Truck_20_LP_40_50_10_20rep.csv     Max            62918

[240 rows x 3 columns]


In [41]:
original_df[original_df['File_name'] == 'prev_Truck_20_LP_0_0_100_1rep.csv']

,File_name,Type,Objective_value
0,prev_Truck_20_LP_0_0_100_1rep.csv,Min,63800
4,prev_Truck_20_LP_0_0_100_1rep.csv,Middle,98400
8,prev_Truck_20_LP_0_0_100_1rep.csv,Max,119600


In [42]:
# 'File_name'과 'Type'이 같은 행끼리 병합
merged_df = pd.merge(original_df, cost_type_df, on=['File_name', 'Type'], how='inner')
print(merged_df)

                               File_name    Type  Objective_value    C_max  \
0      prev_Truck_20_LP_0_0_100_1rep.csv     Min            63800  427.120   
1       now_Truck_20_LP_0_0_100_1rep.csv     Min            68000  436.192   
2     prev_Truck_20_LP_40_50_10_1rep.csv     Min            34751  413.552   
3      now_Truck_20_LP_40_50_10_1rep.csv     Min            34924  451.584   
4      prev_Truck_20_LP_0_0_100_1rep.csv  Middle            98400  466.880   
..                                   ...     ...              ...      ...   
235   now_Truck_20_LP_40_50_10_20rep.csv  Middle            48319  458.872   
236   prev_Truck_20_LP_0_0_100_20rep.csv     Max           112800  490.896   
237    now_Truck_20_LP_0_0_100_20rep.csv     Max           115600  490.912   
238  prev_Truck_20_LP_40_50_10_20rep.csv     Max            59834  480.576   
239   now_Truck_20_LP_40_50_10_20rep.csv     Max            62918  505.376   

        C_avg  
0    384.7000  
1    390.1900  
2    382.6468  

In [43]:
merged_df[merged_df['File_name'] == 'prev_Truck_20_LP_0_0_100_1rep.csv']

,File_name,Type,Objective_value,C_max,C_avg
0,prev_Truck_20_LP_0_0_100_1rep.csv,Min,63800,427.120,384.7000
4,prev_Truck_20_LP_0_0_100_1rep.csv,Middle,98400,466.880,428.1868
8,prev_Truck_20_LP_0_0_100_1rep.csv,Max,119600,521.536,452.7484


In [45]:
# dataframe to csv
merged_df.to_csv('correlation_data_with_C_max.csv', index=False)